In [28]:
import pandas as pd
import numpy as np

# Filter all warnings. If you would like to see the warnings, please comment the two lines below.
import warnings
warnings.filterwarnings('ignore')

In [24]:
def answer_two():
        #Load xls file 
    xls_file = pd.ExcelFile('assets/Energy Indicators.xls')
    energy = xls_file.parse(skiprows=17,skip_footer=(38))
    energy = energy[['Unnamed: 1','Petajoules','Gigajoules','%']]
    # Rename the columns
    energy.columns = ['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']
    # Convert "..." to np.NaN
    energy[['Energy Supply', 'Energy Supply per Capita', '% Renewable']] =  energy[['Energy Supply', 'Energy Supply per Capita', '% Renewable']].replace('...',np.NaN).apply(pd.to_numeric)
    #Conevrt Energy Supply to gigajoules (there are 1,000,000 gigajoules in a petajoule)
    energy['Energy Supply'] = energy['Energy Supply']*1000000
    #Rename Countries
    energy['Country'] = energy['Country'].replace({"Republic of Korea": "South Korea", "United States of America": "United States",
                                                   "United Kingdom of Great Britain and Northern Ireland": "United Kingdom", 
                                                   "China, Hong Kong Special Administrative Region": "Hong Kong"})
    #Remove parantheses 
    energy['Country'] = energy['Country'].str.replace(r" \(.*\)","")
    
    #GDP read_csv & Skip header & only include 10 last years 
    GDP = pd.read_csv('assets/world_bank.csv', skiprows=4)
    #Rename Countries
    GDP = GDP[['Country Name','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015']]
    GDP['Country Name'] = GDP['Country Name'].replace({"Korea, Rep.": "South Korea", "Iran, Islamic Rep.": "Iran", 
                                             "Hong Kong SAR, China": "Hong Kong"})
    #Only inlcude 10 last years of GDP 
    GDP.columns = ['Country','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015']
    #Load scimagojr-3.xlsx
    ScimEn = pd.read_excel(io='scimagojr-3.xlsx')
        
    #Join the three DataFrames on Country
    ScimEn_Energy = pd.merge(ScimEn, energy, how='inner', left_on='Country', right_on='Country')
    df_joint = pd.merge(ScimEn_Energy, GDP, how='inner', left_on='Country', right_on='Country')
    df_joint = df_joint.set_index('Country')
    df_joint_top = df_joint[:15]
    
    #The Actual answer is this, but (I guess) since the df_joint and df_joint_top were defined in the function answer_one() they 
    # were not defined here so I repeated all the code lines, But I think there should be something like 'global' to use the 
    # variables defined in other functions
    return len(df_joint.index) - len(df_joint_top.index)

In [30]:
answer_two()

158

In [29]:
def answer_two():
    # YOUR CODE HERE
    Energy = pd.read_excel('assets/Energy Indicators.xls', header = 15 , skipfooter = 38)
    Energy.drop(columns = ['Unnamed: 0', 'Unnamed: 1'], inplace = True)
    Energy = Energy.rename(columns ={'Unnamed: 2': 'Country', 'Unnamed: 3': 'Energy Supply', 
                            'Unnamed: 4':'Energy Supply per Capita', 'Unnamed: 5':'% Renewable'}).drop([0,1] ,axis =0)
    Energy['Energy Supply'] = Energy['Energy Supply'].replace('...',np.NaN).apply(lambda x: x* 1000000)
    Energy['Country'] = Energy['Country'].str.extract('([^\d\(\)]+)', expand=False).str.strip()
    Energy.replace({'Country':{"Republic of Korea": "South Korea", 
                           "United States of America": "United States", 
                           "United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
                           "China, Hong Kong Special Administrative Region": "Hong Kong"}},inplace =True)
    
    GDP = pd.read_csv('assets/world_bank.csv', header = 4)
    GDP.replace({'Country Name':{"Korea, Rep.": "South Korea",  "Iran, Islamic Rep.": "Iran", "Hong Kong SAR, China": "Hong Kong"}}, inplace =True)
    GDP.drop(columns = GDP.loc[:,'Country Code':'2005'], inplace =True)
    GDP=GDP.rename(columns = {'Country Name':'Country'})
    ScimEn = pd.read_excel('assets/scimagojr-3.xlsx')
    
    from functools import reduce
    mergedata = [ScimEn,Energy,GDP]
    findata = reduce(lambda left, right: pd.merge(left, right, on=['Country'], how = 'inner'), mergedata)
    findata.set_index('Country', inplace =True)
    
    from functools import reduce
    mergedata = [ScimEn,Energy,GDP]
    outdata = reduce(lambda left, right: pd.merge(left, right, on=['Country'], how = 'outer'), mergedata)
    outdata.set_index('Country', inplace =True)
    
    return int(len(outdata.index)-len(findata.index))